In [7]:
import pandas as pd
df = pd.read_csv("lemma/fr-lemmatizer.dict",delimiter="\t")
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
df.to_csv("lemma/fr-lemmatizer-processed.dict", sep='\t',encoding='utf-8',index=False)


In [23]:
import pandas as pd
df = pd.read_csv("lemma/index-vocab/translation_table.csv.2433",encoding='utf-8',delimiter=";",low_memory=False)
df2 = pd.read_csv("lemma/index-vocab/item_model_make.csv.2433",encoding='utf-8',delimiter=";",low_memory=False)

# selecting rows based on condition
rslt_df = df[df['lang'] == 'FR']

#lang;desc;id;text;flag;version  
#df2=rslt_df.drop(['lang','desc','id','flag','version'], axis=1)
#print(df2)
#df2.to_csv("lemma/vocab-1-fr.dict", sep='\t',encoding='utf-8',index=False)
#result = pd.concat(frames)
# Using pandas.concat to extend one column to multiple columns
result = pd.concat([rslt_df['text'],df2['make']])
print(result)
result.to_csv("lemma/vocab-1-fr.dict", sep='\t',encoding='utf-8',index=False)

2907            Ø flexible
2908            Ø flexible
2909            Ø flexible
2910            Ø flexible
2911            Ø flexible
                ...       
3379728    Massey Ferguson
3379729             Unimog
3379730              Volvo
3379731        New Holland
3379732        Dana-Spicer
Length: 3776356, dtype: object


In [26]:
import re
from collections import Counter
words = re.findall(r'\w+', open('lemma/vocab-1-fr.dict',encoding='utf-8').read().lower())
fr_vocab = Counter(words)
fr_vocab.most_common(10)


[('deere', 428988),
 ('john', 428956),
 ('massey', 419972),
 ('ferguson', 417673),
 ('claas', 330169),
 ('new', 289660),
 ('holland', 289542),
 ('case', 249700),
 ('ih', 248648),
 ('harvesting', 239785)]

In [39]:
fr_vocab_lemmas = {spicy_stemmer.stem_word(word):fr_vocab[word] for word in list(fr_vocab.keys()) if len(word)>2}


In [42]:
import csv
with open('lemma/wordstat-fr.txt', 'w', encoding='utf-8') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)      
    write.writerows(list(map(list, fr_vocab.items())))
print(len(fr_vocab.keys()))

with open('lemma/wordlemma-stat-fr.txt', 'w', encoding='utf-8') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)      
    write.writerows(list(map(list, fr_vocab_lemmas.items())))
print(len(fr_vocab_lemmas.keys()))

47838
42976


## Read vocabs from solr 

In [3]:
from collections import defaultdict

with open('lemma/wordlemma-stat-fr.csv', 'r', encoding="utf8") as file:
    d = {line.rstrip('\n').split(',')[0]:int(line.rstrip('\n').split(',')[1]) for line in file.readlines()}
    fr_vocab_lemmas = defaultdict(lambda:0, d)
    print(fr_vocab_lemmas['a'])
    print(fr_vocab_lemmas['ferguson'])

0
417673


In [4]:
from solr_client import SolrClient
solr1 = SolrClient('localhost',8986)
solr2 = SolrClient('be-plw-tst-0050',8983)
solr2.auth("solr","SolrRocks")
solr1.auth("solr","SolrRocks")
#solr2 = SolrClient('localhost',8981)
solr2.search('textsearch', 'STARTVEILIGHEIDSSCHAKELAAR','nl')

{'numFound': 1, 'ids': ['50554053']}

In [5]:
import os

#root = 'queries'
#root = 'top_mbf'
#root = 'hybrid_syn'
root = 'french'
#root="french_plural"
start_records=0
max_records = 3000
#root = 'top_mbf'
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(root):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames if file.strip().endswith('.txt') or file.strip().endswith('.csv')]

datasets = {}
for f in listOfFiles:
    filename = os.path.basename(f)
    queries = []
    with open(f, 'r', encoding="utf8") as file:
        lines = [line.rstrip('\n') for line in file.readlines() if not line.startswith('#')]
        for line in lines[start_records:max_records]:
            queries.extend([phrase.split('=')[0] for phrase in line.split(',') if not 'filter_' in phrase and not '\\' in phrase])
    datasets[filename] = queries
for filename in datasets.keys():
    print(filename, datasets[os.path.basename(filename)][0:10])


type1_mbf_fr.csv ['durite', 'siege', 'batterie', 'dewalt', 'al', 'peinture', 'echappement', 'rotule', 'courroie', 're']
type2_mbf_fr.csv ['camera', 'cable accelerateur', 'contacteur de demarrage', 'tube echappement', 'contacteur demarrage', 'cable embrayage', 'cle dynamometrique', 'cameras', 'camera de recul', 'cable demarrage']


In [14]:
import functools
import re
sum1=0
sum2=0

for filepath in listOfFiles:
    file = os.path.basename(filepath)
    lang = file.split('.')[0].split('_')[-1]
    if not lang in ['en', 'fr', 'nl','de']:
        lang='en'
    print(file, lang)
    queries = datasets[file]
    for text in queries:  
        q = re.sub(' +', ' ', text)
        lemmas=toy_stemmer.stem_text(q)
        spicyStems=spicy_stemmer.stem_text(q)
        
        for word in q.strip().split(' '):     
            lemma = lemmas[word]
            spicyStem=spicy_stemmer.stem_word(word)
            if word in spicyStems.keys():
                spicyStem=spicyStems[word]
            porterStem=porterFr_stemmer.stem_word(word)
            lemmaStem=porterFr_stemmer.stem_word(lemma)
            if len(q)>2 and not lemmaStem == porterStem and spicyStem in fr_vocab_lemmas.keys():
                solr_q_result=['','','','']
                if (fr_vocab_lemmas[spicyStem]>100):
                # check solr results comparison
                    res1 = solr2.search('textsearch7', q, 'fr')
                    res2 = solr2.search('textsearch6', q, 'fr')
                    if abs(res1['numFound'] - res2['numFound'])/max(res1['numFound'], res2['numFound'],1)>0.01:
                        #res_q=solr2.search('textsearch7', q, 'fr')
                        solr_q_result=[str(res1['numFound']),str(res2['numFound']),'','HIGH']
                        #print(q, res1['numFound'], res2['numFound'])
                    sum1+=res1['numFound']
                    sum2+=res2['numFound']

                print(",".join([q,word, porterStem,lemma,str(fr_vocab_lemmas[spicyStem])] + solr_q_result))
            #if not functools.reduce(lambda x, y: x and y, map(lambda a, b: a == b, list1, list2), True):
                #print("Both List are same")
            #else:
print(sum1,sum2)


type1_mbf_fr.csv fr
ressort,ressort,resort,ressortir,2794,,,,
joint,joint,joint,joindre,1,,,,
masse,masse,mase,mer,18,,,,
joint torique,joint,joint,joindre,1,,,,
vis,vis,vis,vivre,11,,,,
joint cuivre,joint,joint,joindre,1,,,,
prise,prise,pris,prendre,43,,,,
balai,balai,balai,baler,200,372,352,,HIGH
cha,cha,cha,cher,2,,,,
joint injecteur,joint,joint,joindre,1,,,,
servante,servante,servant,serviteur,140,,,,
joint ,joint,joint,joindre,1,,,,
essuie glace,essuie,esu,essuyer,521,193,188,,HIGH
masse avant,masse,mase,mer,18,,,,
balais,balais,balai,baler,200,372,352,,HIGH
optima,optima,optima,optimum,7,,,,
la buvette,la,,ler,31,,,,
tuyaux,tuyaux,tuyal,tuyau,122,,,,
relai,relai,relai,reler,21,,,,
bellota,bellota,belota,belloter,4,,,,
vis creuse,vis,vis,vivre,11,,,,
vis creuse,creuse,creu,creuser,84,,,,
joint echappement,joint,joint,joindre,1,,,,
prise de force,prise,pris,prendre,43,,,,
passe cloison,passe,pase,per,297,,,,
joint toriques,joint,joint,joindre,1,,,,
joints,joints,joint,joindre,1389,

camera san,camera,camera,camer,85,,,,
ressort pedale emb,ressort,resort,ressortir,2794,,,,
ressort rappel pedale,ressort,resort,ressortir,252,,,,
kit camera sans fil,camera,camera,camer,1,,,,
kit camera sans fil,sans,,san,2113,,,,
ecran camera,camera,camera,camer,85,,,,
camera cable,camera,camera,camer,85,,,,
tube de prise de force,prise,pris,prendre,43,,,,
camera rec,camera,camera,camer,1,,,,
ressort case pedale,ressort,resort,ressortir,252,,,,
ressort pedale de frein tracteur ih,ressort,resort,ressortir,2794,,,,
ressort pedale ,ressort,resort,ressortir,2794,,,,
camera sur,camera,camera,camer,85,,,,
camera de recul sans fil,sans,,san,2113,,,,
turbo kubota,kubota,kubota,kuboter,21563,,,,
cable de camera,camera,camera,camer,85,,,,
decapant soudure,decapant,decapant,decaper,1,,,,
cable kubota,kubota,kubota,kuboter,21563,,,,
champ magnetique,champ,champ,champagne,1,,,,
support camera,camera,camera,camer,85,,,,
camera sa,camera,camera,camer,85,,,,
cable embrayage prise de force,prise,pris,

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

# Stemmers

In [4]:
!python -m spacy download nl_core_news_sm
!python -m spacy download fr_core_news_md

You should consider upgrading via the 'c:\Projects\tvh-search-labeling\venv\Scripts\python.exe -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')


In [1]:
import spacy
from solr_client import SolrClient
from stemmers import Stemmer
from stemmers import SpicyStemmer
from stemmers import TruncateStem


    
solr1 = SolrClient('localhost',8986)

#French stemmers
toy_stemmer = Stemmer(solr1, index = 'textsearch', fieldtype='query', field = 'text_nlp_fr_z', name = 'FR nlp')
lightFr_stemmer = Stemmer(solr1, index = 'textsearch', fieldtype='query', field = 'text_test_fr_z', name = 'FR LightFr')
porterFr_stemmer = Stemmer(solr1, index = 'textsearch', fieldtype='fieldvalue', field = 'text_test_fr_z', name = 'FR Porter')
spicy_stemmer = SpicyStemmer('fr_core_news_md', name = 'FR spicy')

# Dutch stemmers
porterNL_stemmer = Stemmer(solr1, index = 'textsearch', fieldtype='query', field = 'text_test_nl_z', name='NL Porter')
spicyNL_stemmer = SpicyStemmer('nl_core_news_sm', name='NL spicy')
nl_stemmer = Stemmer(solr1, index = 'textsearch', fieldtype='query', field = 'text_nlp_nl_z', name='NL nlp')

# Truncate stemmers (remove last N characters)
trunc_stemmers = [TruncateStem(i) for i in list(range(1, 7))]
print (trunc_stemmers[0].name)

print(toy_stemmer.stem_word("trous"))
print(spicy_stemmer.stem_word("long-métrage"))
print(lightFr_stemmer.stem_word("trous"))
print(porterFr_stemmer.stem_word("long-métrage"))

print(toy_stemmer.stem_text("barre a trous"))
print(spicy_stemmer.stem_text("barre a trous"))

print(porterNL_stemmer.stem_word("accessoires"))
print(spicyNL_stemmer.stem_word("accessoires"))
print(nl_stemmer.stem_word("accessoires"))

#Stemmers to test
#stemmers=[porterNL_stemmer, spicyNL_stemmer, nl_stemmer]
#stemmers=[toy_stemmer, spicy_stemmer, lightFr_stemmer, porterFr_stemmer]
stemmers=[]

stemmers+=trunc_stemmers

TruncateStem1
trou
long
trous
long-metrag
{'barre': 'barre', 'a': 'avoir', 'trous': 'trou'}
{'barre': 'barre', 'a': 'avoir', 'trous': 'trou'}
accessoires
accessoire
accessoire


In [13]:
spicyStems=defaultdict(lambda a:spicy_stemmer.stem_word(a),spicy_stemmer.stem_text("d'alimentation"))
        
spicyStems["d'alimentation"]        
print(spicy_stemmer.stem_text("d'alimentation"))

TypeError: <lambda>() missing 1 required positional argument: 'a'

## Calculate understemming/ overstemming metrics

In [3]:
lang='fr'

with open('concept_groups_{0}.txt'.format(lang), 'r',encoding='utf8') as file:
    concept_groups = [line.rstrip('\n').split(",") for line in file.readlines()]

In [4]:

#concept_groups = concept_groups[0:10]
def GDMT(cg):
    gdmt = 0
    for g in cg:
        ng = len(g)
        gdmt += 0.5 * ng * (ng-1)
    return gdmt

def GDNT(cg):
    gdnt = 0
    W = sum([len(g) for g in cg])
    for g in cg:
        ng = len(g)
        gdnt += 0.5 * ng * (W-ng)
    return gdnt

print ("GDMT = ", GDMT(concept_groups))
print ("GDNT = " , GDNT(concept_groups))

GDMT =  1898.0
GDNT =  8967832.0


In [5]:
from collections import Counter
def GUMT(cg, stemmer, debug=False):
    gumt = 0
    for g in cg:
        if debug:
            print("For group " , repr(g) )
        stems = stemmer.stem_words(g)
        ng = len(g)
        unique_stems = Counter(stems)
        if debug:
            print("Unique stems with their instances are: " ,
             repr(dict(unique_stems)))
        umt = 0
        for unique_stem, count in unique_stems.items():
            umt += 0.5 * count * (ng-count)
        gumt += umt
        if debug:
            print("Unmerged concept pairs in this group = ", umt)

    return gumt

def UI(cg, stemmer):
    return GUMT(cg, stemmer)/GDMT(cg)

for stemmer in stemmers:
    print(stemmer.name, "Understemming Error = %.2f%%" % (UI(concept_groups, stemmer)*100))
#print("nlp GUMT = ", GUMT(concept_groups,toy_stemmer, False))
#print("spicy GUMT = ", GUMT(concept_groups,spicy_stemmer, False))
#print("light GUMT = ", GUMT(concept_groups,lightFr_stemmer, False))
#print("porter GUMT = ", GUMT(concept_groups,porterFr_stemmer, False))


TruncateStem1 Understemming Error = 99.42%
TruncateStem2 Understemming Error = 98.63%
TruncateStem3 Understemming Error = 98.26%
TruncateStem4 Understemming Error = 92.31%


In [12]:
def UI(cg, stemmer):
    return GUMT(cg, stemmer)/GDMT(cg)

for stemmer in stemmers:
    print(stemmer.name, "Understemming Error = %.2f%%" % (UI(concept_groups, stemmer)*100))

#print("Understemming Error = %.2f%%" % (UI(concept_groups, toy_stemmer)*100))
#print("Spicy Understemming Error = %.2f%%" % (UI(concept_groups, spicy_stemmer)*100))
#print("Light French Understemming Error = %.2f%%" % (UI(concept_groups, lightFr_stemmer)*100))
#print("Porter Understemming Error = %.2f%%" % (UI(concept_groups, porterFr_stemmer)*100))

TruncateStem1 Understemming Error = 99.42%
TruncateStem2 Understemming Error = 98.63%
TruncateStem3 Understemming Error = 98.26%
TruncateStem4 Understemming Error = 92.31%
TruncateStem5 Understemming Error = 82.56%
TruncateStem6 Understemming Error = 67.02%


In [6]:
stemmers[2].stem_word('pana')

'p'

In [8]:
from collections import defaultdict
from json import dumps

def stem_groups(cg, stm):
    """
    cg is the concept group
    stm is the stemmer
    """
    stem_group = defaultdict(list)
    stem_group_inverted_cg = defaultdict(list)
    for idx, g in enumerate(cg):
        for word in g:
            stem = stm.stem_word(word)
            stem_group[stem].append(word)
            #if len(stem_group_inverted_cg[stem])>0 and not str(idx) in stem_group_inverted_cg[stem]:
             #   print("overstemming case:",word,stem, ' '.join(str(stem_group_inverted_cg[stem])), str(idx)) 
            stem_group_inverted_cg[stem].append(str(idx))
    return stem_group, stem_group_inverted_cg

#print(dumps(stem_groups(concept_groups, porterFr_stemmer), indent=4))

In [13]:
from collections import Counter
def GWMT(stem_group, stem_group_inverted_cg, debug=False):
    gwmt = 0
    for sg_key in stem_group.keys():
        g = stem_group[sg_key]
        if debug:
            print("For group " , repr(g) )
        ng = len(g)
        unique_stems = Counter( stem_group_inverted_cg[sg_key])
        if debug:
            print("Unique cg with their instances are: " ,
             repr(dict(unique_stems)))
        wmt = 0
        for unique_stem, count in unique_stems.items():
            wmt += 0.5 * count * (ng-count)
        gwmt += wmt
        if debug:
            print("Overmerged concept pairs in this group = ", wmt)

    return gwmt

def WI(cg, stm):
    stem_group, stem_group_inverted_cg = stem_groups(cg,stm)
    return GWMT(stem_group, stem_group_inverted_cg)/GDNT(cg)
for stemmer in stemmers:
    print(stemmer.name, "Overstemming Error = %.4f%%" % (WI(concept_groups,  stemmer)*100))
#print("Overstemming Error = %.4f%%" % (WI(concept_groups,  toy_stemmer)*100))
#print("Spicy Overstemming Error = %.4f%%" % (WI(concept_groups,  spicy_stemmer)*100))
#print("LightFr Overstemming Error = %.4f%%" % (WI(concept_groups,  lightFr_stemmer)*100))
#print("Porter Overstemming Error = %.4f%%" % (WI(concept_groups,  porterFr_stemmer)*100))
#print("Spicy Understemming Error = %.2f%%" % (UI(concept_groups, spicy_stemmer)*100))
#print("Light French Understemming Error = %.2f%%" % (UI(concept_groups, lightFr_stemmer)*100))
#print("Porter Understemming Error = %.2f%%" % (UI(concept_groups, porterFr_stemmer)*100))

TruncateStem1 Overstemming Error = 0.0036%
TruncateStem2 Overstemming Error = 0.0289%
TruncateStem3 Overstemming Error = 0.3008%
TruncateStem4 Overstemming Error = 2.0770%
TruncateStem5 Overstemming Error = 7.6879%
TruncateStem6 Overstemming Error = 18.3915%
